# Attempt At a Wordle Solving Bot

set up necessary imports

In [1]:
!pip install nltk

In [1]:
import random
import nltk
from nltk.corpus import words
from collections import Counter
import math

Load list of words from the .txt file, filter them accordingly into new file containing only common words from nltk for ease of use for this test.

In [107]:
# Download NLTK data
#nltk.download('words')

# Load a list of valid 5-letter words from a file
with open('wordle-word-list.txt', 'r') as file:
    word_list = file.read().splitlines()

# Get the list of common English words from NLTK
nltk_words = set(words.words())

# Filter the word list to include only common words
filtered_word_list = [word for word in word_list if word in nltk_words]

# Save the filtered word list to a new file
with open('filtered-wordle-word-list.txt', 'w') as file:
    for word in filtered_word_list:
        file.write(f"{word}\n")

print(f"Filtered word list saved. Original list had {len(word_list)} words, filtered list has {len(filtered_word_list)} words.")

solution = random.choice(word_list)

Filtered word list saved. Original list had 14855 words, filtered list has 6111 words.


## Feedback

Feedback function for displaying the progress of the game. Currently this will be used for both human and bot based feedback reports. 
Simple GUI based on the worlde gameboard to allow for better visibility.

In [108]:
def get_feedback(guess, solution):
    feedback = [''] * 5
    solution_chars = list(solution)
    guess_chars = list(guess)
    
    # First pass: find correct letters (🟩)
    for i in range(5):
        if guess_chars[i] == solution_chars[i]:
            feedback[i] = '🟩'
            solution_chars[i] = None  # Remove matched character to avoid duplicates
    
    # Second pass: find misplaced letters (🟨)
    for i in range(5):
        if feedback[i] == '':
            if guess_chars[i] in solution_chars:
                feedback[i] = '🟨'
                solution_chars[solution_chars.index(guess_chars[i])] = None
            else:
                feedback[i] = '⬛'  # Incorrect letters (⬛)
    
    return ''.join(feedback)

## Play the wordle game, USER BASED, !!NOT!! BOT

In [ ]:
def play_wordle(solution):
    guesses = []
    while len(guesses) < 6:
        guess = input("Enter your guess: ").lower()
        if len(guess) != 5:
            print("Guess must be exactly 5 letters long.")
            continue
        feedback = get_feedback(guess, solution)
        print(f"Feedback: {feedback}")
        guesses.append((guess, feedback))
        if guess == solution:
            print("Congratulations! You guessed the word!")
            break
    else:
        print(f"Sorry, you did not guess the word. The correct word was: {solution}")
    return guesses

# Play the game
solution = random.choice(filtered_word_list)
guesses = play_wordle(solution)

## Bot Guess Logic 

Currently undefined and using a brute force method in which random guesses are given and no learning is used.

## Brute Force ##

In [109]:
def bot_brute_guess(word_list, feedback_history):
    return random.choice(word_list)

## Simple Filter

In [110]:
def bot_guess_simple(word_list, feedback_history):
    # Filter words based on previous feedback
    possible_words = word_list
    for guess, feedback in feedback_history:
        new_possible_words = []
        for word in possible_words:
            if get_feedback(guess, word) == feedback:
                new_possible_words.append(word)
        possible_words = new_possible_words
    return random.choice(possible_words) if possible_words else random.choice(word_list)

## Frequency Based

In [111]:
def bot_guess_frequency(word_list, feedback_history):
    # Filter words based on previous feedback
    possible_words = word_list
    for guess, feedback in feedback_history:
        new_possible_words = []
        for word in possible_words:
            if get_feedback(guess, word) == feedback:
                new_possible_words.append(word)
        possible_words = new_possible_words
    
    # Calculate letter frequencies
    letter_frequencies = Counter(''.join(possible_words))
    
    # Score words based on letter frequencies
    def score_word(word):
        return sum(letter_frequencies[letter] for letter in set(word))
    
    best_guess = max(possible_words, key=score_word)
    return best_guess if possible_words else random.choice(word_list)

## Custom Algorithm Based off My Approach

In [112]:
def bot_custom_guess(word_list, feedback_history):
    # Define the initial guess with multiple vowels
    initial_vowel_guess = "audio"  # Can be replaced with any word with multiple vowels

    if not feedback_history:
        return initial_vowel_guess

    # Track letters based on feedback
    correct_positions = {}
    misplaced_letters = set()
    incorrect_letters = set()
    must_have_letters = set()
    excluded_positions = {i: set() for i in range(5)}
    guessed_words = set(guess for guess, _ in feedback_history)  # Track guessed words
    known_repeats = set()  # Track letters known to repeat

    for guess, feedback in feedback_history:
        char_count = {}
        for i, (char, fb) in enumerate(zip(guess, feedback)):
            if fb == '🟩':
                correct_positions[i] = char
                must_have_letters.add(char)
            elif fb == '🟨':
                misplaced_letters.add(char)
                excluded_positions[i].add(char)
                must_have_letters.add(char)
            elif fb == '⬛':
                # Only add to incorrect_letters if char is not in correct_positions or misplaced_letters
                if char not in correct_positions.values() and char not in misplaced_letters:
                    incorrect_letters.add(char)
            char_count[char] = char_count.get(char, 0) + 1

        # Check for known repeats
        for char, count in char_count.items():
            if count > 1 and any(fb in ['🟩', '🟨'] for j, fb in enumerate(feedback) if guess[j] == char):
                known_repeats.add(char)

    print(f"Correct positions: {correct_positions}")
    print(f"Misplaced letters: {misplaced_letters}")
    print(f"Incorrect letters: {incorrect_letters}")
    print(f"Excluded positions: {excluded_positions}")
    print(f"Must have letters: {must_have_letters}")
    print(f"Guessed words: {guessed_words}")
    print(f"Known repeats: {known_repeats}")

    # Filter possible words
    def word_matches_feedback(word):
        if word in guessed_words:
            return False
        for i, char in enumerate(word):
            # Exclude words with incorrect letters
            if char in incorrect_letters:
                return False
            # Ensure correct positions (🟩)
            if i in correct_positions and word[i] != correct_positions[i]:
                return False
            # Ensure misplaced letters (🟨) are in the word but not in the excluded positions
            if char in misplaced_letters and word[i] in excluded_positions[i]:
                return False
        # Ensure all must-have letters (🟩 and 🟨) are present in the word
        if not all(char in word for char in must_have_letters):
            return False
        return True

    possible_words = [word for word in word_list if word_matches_feedback(word)]

    # Prefer words without repeated characters unless known repeats are present
    def word_has_repeats(word):
        return len(set(word)) < len(word)

    if known_repeats:
        possible_words = [word for word in possible_words if any(char in known_repeats for char in word)]
    else:
        possible_words = [word for word in possible_words if not word_has_repeats(word)]

    print(f"Filtered words: {possible_words}")

    # If no word is found, return a random word from the original list
    if possible_words:
        return random.choice(possible_words)
    else:
        # As a fallback, return a random word from the original list if filtering is too strict
        return random.choice(word_list)


## Wordle Bot Rules and Function. 

In [113]:
def play_wordle_bot(solution, word_list, bot_guess_function):
    guesses = []
    feedback_history = []
    while len(guesses) < 6:
        guess = bot_guess_function(word_list, feedback_history)
        print(f"Bot guess: {guess}")  # Debugging print
        if len(guess) != 5:
            print("Invalid guess length, skipping...")  # Debugging print
            continue  # Ensure the bot guess is exactly 5 letters long
        feedback = get_feedback(guess, solution)
        print(f"Feedback: {feedback}")  # Debugging print
        feedback_history.append((guess, feedback))
        guesses.append((guess, feedback))
        if guess == solution:
            print(f"Bot guessed the word! Correct word was: {solution}")
            break
    else:
        print(f"Bot did not guess the word. The correct word was: {solution}")
    return guesses

## Execute Bots and Comparison

In [124]:
# Define the strategies
strategies = {
    'brute': bot_brute_guess,
    'simple': bot_guess_simple,
    'frequency': bot_guess_frequency,
    'custom': bot_custom_guess
}

# Play the game with each strategy and record the results
results = {}
for strategy_name, strategy_function in strategies.items():
    print(f"\nRunning strategy: {strategy_name}")
    solution = random.choice(word_list)
    guesses = play_wordle_bot(solution, word_list, strategy_function)
    results[strategy_name] = len(guesses)

# Print the results
for strategy_name, num_guesses in results.items():
    print(f"Strategy {strategy_name} solved the puzzle in {num_guesses} guesses")


Running strategy: brute
Bot guess: kiore
Feedback: ⬛⬛🟨⬛⬛
Bot guess: crimp
Feedback: ⬛⬛⬛🟨🟨
Bot guess: kanas
Feedback: ⬛⬛⬛⬛🟨
Bot guess: purpy
Feedback: 🟨⬛⬛⬛🟩
Bot guess: toses
Feedback: ⬛🟩🟨⬛⬛
Bot guess: ducal
Feedback: ⬛⬛⬛⬛⬛
Bot did not guess the word. The correct word was: mopsy

Running strategy: simple
Bot guess: thees
Feedback: ⬛⬛🟩⬛⬛
Bot guess: pleno
Feedback: ⬛⬛🟩⬛🟩
Bot guess: grebo
Feedback: 🟩🟩🟩⬛🟩
Bot guess: grego
Feedback: 🟩🟩🟩🟩🟩
Bot guessed the word! Correct word was: grego

Running strategy: frequency
Bot guess: aeros
Feedback: ⬛🟨⬛⬛⬛
Bot guess: eldin
Feedback: 🟨🟨⬛⬛⬛
Bot guess: muley
Feedback: ⬛⬛🟩🟩🟨
Bot guess: hyleg
Feedback: 🟩🟩🟩🟩🟩
Bot guessed the word! Correct word was: hyleg

Running strategy: custom
Bot guess: audio
Feedback: 🟨⬛⬛⬛⬛
Correct positions: {}
Misplaced letters: {'a'}
Incorrect letters: {'u', 'd', 'o', 'i'}
Excluded positions: {0: {'a'}, 1: set(), 2: set(), 3: set(), 4: set()}
Must have letters: {'a'}
Guessed words: {'audio'}
Known repeats: set()
Filtered words: ['bach